## Importing required libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [2]:
import requests

## Setting maxcolwidth to 800 for readability

In [3]:
pd.set_option('max_colwidth', 800)

## instantiating BeautifulSoup object and reading the table from wikipedia page

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

## Creating a csv_writer to append the scraped content in an xlx file by initally defining column names 

In [5]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

## Main Scraping of data begins here

In [64]:
table = soup.find('table', class_ = 'wikitable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)


ValueError: I/O operation on closed file.

## Closing the CSV file

In [10]:
csv_file.close()

## Creating a pandas dataframe

In [167]:
toronto_df=pd.read_csv('toronto_postal_codes.csv')

## Gauging the shape of the created pandas dataframe

In [168]:
toronto_df.shape

(180, 3)

In [169]:
toronto_df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A\r\n,Not assigned\r\n,NaN
1,M2A\r\n,Not assigned\r\n,NaN
2,M3A\r\n,North York\r\n,Parkwoods
3,M4A\r\n,North York\r\n,Victoria Village
4,M5A\r\n,Downtown Toronto\r\n,Regent Park / Harbourfront


In [170]:
toronto_df=toronto_df.replace('\r\n', '',regex=True)

In [171]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [172]:
toronto_df_copy=toronto_df

In [179]:
drop_index=toronto_df.loc[toronto_df['Borough']=='Not assigned'].index

In [180]:
drop_index

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [187]:
toronto_df_copy.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [194]:
toronto_df_copy.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [195]:
toronto_df_copy.reset_index(inplace=True)

In [200]:
toronto_df_copy.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [199]:
toronto_df_copy=toronto_df_copy.drop('index',1)

In [224]:
toronto_df_copy['Neighbourhood'].replace('/',',',regex=True,inplace=True)

In [216]:
t=toronto_df_copy[['Neighbourhood']]
t.replace('/',',',regex=True,inplace=True)

In [226]:
toronto_df_copy.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [221]:
toronto_df_copy.shape

(103, 3)